In [1]:
import json
import urllib
import math
import pandas as pd

In [2]:
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # semi-major axis
ee = 0.00669342162296594323  # Eccentricity squared

In [3]:
'''
输入（经度，维度）
'''
def bd09_to_gcj02(bd_lon, bd_lat):
    """
    百度坐标系(BD-09)转火星坐标系(GCJ-02)
    百度——>谷歌、高德
    :param bd_lat:百度坐标纬度
    :param bd_lon:百度坐标经度
    :return:转换后的坐标列表形式
    """
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]
def gcj02_to_wgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    if out_of_china(lng, lat):
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]
def bd09_to_wgs84(bd_lon, bd_lat):
    lon, lat = bd09_to_gcj02(bd_lon, bd_lat)
    return gcj02_to_wgs84(lon, lat)
def bd09_to_wgs84(bd_lon, bd_lat):
    lon, lat = bd09_to_gcj02(bd_lon, bd_lat)
    return gcj02_to_wgs84(lon, lat)
def gcj02_to_bd09(lng, lat):
    """
    火星坐标系(GCJ-02)转百度坐标系(BD-09)
    谷歌、高德——>百度
    :param lng:火星坐标经度
    :param lat:火星坐标纬度
    :return:
    """
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_pi)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_pi)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng, bd_lat]
def wgs84_to_gcj02(lng, lat):
    """
    WGS84转GCJ02(火星坐标系)
    :param lng:WGS84坐标系的经度
    :param lat:WGS84坐标系的纬度
    :return:
    """
    if out_of_china(lng, lat):  # 判断是否在国内
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]
def wgs84_to_bd09(lon, lat):
    lon, lat = wgs84_to_gcj02(lon, lat)
    return gcj02_to_bd09(lon, lat)

def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret
def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret

————————————————

The script above is referenced from https://blog.csdn.net/weixin_38468077/article/details/115689448

Data can be downloaded from https://drive.google.com/drive/folders/1GOhMd0UbGnod2Kgtyw5k8_3KYFlEBHT0?usp=sharing

In [4]:
poi = pd.read_csv('Data/accessibility/OD_54km.csv', encoding='utf-8')

In [5]:
poi.head()

,IN_FID,NEAR_FID,NEAR_DIST,NEAR_RANK,序号,机构名,机构地,行政区,机构类,机构级,经营性,床位数,牙椅数,wgs_lng,wgs_lat,POP,lng_84,lon_84
0,0,410,356.067221,1,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,三级,非营利性,990,13,113.252797,23.133996,169233.2473,113.249357,23.133543
1,0,411,1649.255304,2,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,三级,非营利性,990,13,113.252797,23.133996,120239.8201,113.268876,23.133263
2,0,444,1975.860208,3,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,三级,非营利性,990,13,113.252797,23.133996,100080.9571,113.249658,23.151595
3,0,385,2085.556082,4,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,三级,非营利性,990,13,113.252797,23.133996,150382.4975,113.249056,23.115490
4,0,409,2352.344506,5,A1,广州市第一人民医院,广州市盘福路1号,越秀区,综合医院,三级,非营利性,990,13,113.252797,23.133996,194210.8505,113.229837,23.133820


In [6]:
poi.dropna(inplace=True)

In [7]:
poi.isna().sum()

IN_FID       0
NEAR_FID     0
NEAR_DIST    0
NEAR_RANK    0
序号           0
机构名          0
机构地          0
行政区          0
机构类          0
机构级          0
经营性          0
床位数          0
牙椅数          0
wgs_lng      0
wgs_lat      0
POP          0
lng_84       0
lon_84       0
dtype: int64

In [9]:
poi[['gcj_lng_D', 'gcj_lat_D']] = poi.apply(lambda row: pd.Series(wgs84_to_gcj02(row['wgs_lng'], row['wgs_lat'])), axis=1)
poi[['gcj_lng_O', 'gcj_lat_O']] = poi.apply(lambda row: pd.Series(wgs84_to_gcj02(row['lng_84'], row['lon_84'])), axis=1)

In [11]:
poi.to_csv('Data/accessibility/OD_54km_gcj.csv', index=False, encoding='utf-8')